In [63]:
import numpy as np
import sys
import sympy as sym

class manipulator():

  def __init__(self, no_of_links, type_of_joints, dh_params):
    super().__init__()
    self.no_of_links = no_of_links
    self.type_of_joints = list(type_of_joints)
    self.dh_params = dh_params
    if len(self.dh_params) != self.no_of_links:
      print("ERROR: Please make sure the number of rows in the DH parameters provided is equal to the number of links.")
      sys.exit()
    self.dh_matrices = []
    for i in range(0, self.no_of_links):    
      temp = self.dh_params[i]
      theta = temp[0]
      d = temp[1]
      a = temp[2]
      alpha = temp[3]
      A1 = np.array([[np.cos(theta), -np.sin(theta), 0, 0],[np.sin(theta), np.cos(theta), 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
      A2 = np.array([[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, d],[0, 0, 0, 1]])
      A3 = np.array([[1, 0, 0, a],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
      A4 = np.array([[1, 0, 0, 0],[0, np.cos(alpha), -np.sin(alpha), 0],[0, np.sin(alpha), np.cos(alpha), 0],[0, 0, 0, 1]])
      A_final = A1@A2@A3@A4
      self.dh_matrices.append(A_final)
    self.dh_matrices = np.array(self.dh_matrices)

    self.transform_matrices = []
    self.rotation_matrices = []

    for i in range(1, self.no_of_links+1):
      temp = self.dh_matrices[0]
      for j in range(1, i):
        temp = temp@self.dh_matrices[j]
      rot_temp = temp
      rot_temp = np.delete(rot_temp, (3), axis = 0)
      rot_temp = np.delete(rot_temp, (3), axis = 1)
      self.transform_matrices.append(temp)
      self.rotation_matrices.append(rot_temp)

    self.transform_matrices = np.array(self.transform_matrices)
    self.rotation_matrices = np.array(self.rotation_matrices)

    self.end_effector_posn = np.delete(self.transform_matrices[len(self.transform_matrices)-1]@np.array([[0],[0],[0],[1]]), (3), axis = 0)

    self.z = []
    self.origins = []
    self.z.append(np.array([[0],[0],[1]]))
    self.origins.append(np.array([[0],[0],[0]]))

    for i in range(0, self.no_of_links):
      temp_z = self.rotation_matrices[i]@np.array([[0],[0],[1]])
      self.z.append(temp_z)
      temp_origin = self.transform_matrices[i]@np.array([[0],[0],[0],[1]])
      temp_origin = np.delete(temp_origin, (3), axis = 0)
      self.origins.append(temp_origin)
    
    self.z = np.array(self.z)
    self.origins = np.array(self.origins)  
    self.jacobian = []
    for i in range(0,self.no_of_links):
      if self.type_of_joints[i] == "R":
        temp_1 = np.cross(self.z[i], self.end_effector_posn-self.origins[i], axis = 0)
        temp_2 = self.z[i]
      if self.type_of_joints[i] == "P":
        temp_1 = self.z[i]
        temp_2 = np.array([[0],[0],[0]])
      temp_jacobian = np.vstack((temp_1, temp_2))
      self.jacobian.append(temp_jacobian)
    self.jacobian = np.array(self.jacobian)
    self.jacobian = self.jacobian.reshape((self.no_of_links,6))
    self.jacobian = np.transpose(self.jacobian)
    self.Jv = np.delete(self.jacobian,(3,4,5), axis = 0)

    print("End Effector Position:")
    print(self.end_effector_posn)
    print("Manipulator Jacobian:")
    print(self.jacobian)
    print("Velocity Jacobian (can be multiplied with qdot to obtain end-effector velocity):")
    print(self.Jv)

my_robot = manipulator(3, "RRR", [[np.pi/3,0,1,0],[0,0,0,np.pi/2],[0,0,0,np.pi/2]])
print("STANFORD MANIPULATOR:")
stanford_manipulator = manipulator(3, "RRP", [[0,0,0,-np.pi/2],[0,2,0,np.pi/2],[0,2,0,0]])
print("SCARA MANIPULATOR:")
SCARA = manipulator(3, "RRP", [[0,0,2,0],[0,0,2,np.pi],[0,2,0,0]])
print("PPP MANIPULATOR:")
PPP = manipulator(3, "PPP", [[0,1,0,-np.pi/2],[-np.pi/2,2,0,-np.pi/2],[0,3,0,0]])
print("6R MANIPULATOR:")
R6 = manipulator(6, "RRRRRR", [[np.pi/2,1,0,-np.pi/2],[0,0,1,0],[0,0,1,np.pi/2],[0,0,0,np.pi/2],[0,0,0,-np.pi/2],[0,1,0,0]])

End Effector Position:
[[0.5      ]
 [0.8660254]
 [0.       ]]
Manipulator Jacobian:
[[-8.66025404e-01  0.00000000e+00 -0.00000000e+00]
 [ 5.00000000e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  8.66025404e-01]
 [ 0.00000000e+00  0.00000000e+00 -5.00000000e-01]
 [ 1.00000000e+00  1.00000000e+00  6.12323400e-17]]
Velocity Jacobian (can be multiplied with qdot to obtain end-effector velocity):
[[-0.8660254  0.        -0.       ]
 [ 0.5        0.         0.       ]
 [ 0.         0.         0.       ]]
STANFORD MANIPULATOR:
End Effector Position:
[[0.]
 [2.]
 [2.]]
Manipulator Jacobian:
[[-2.000000e+00  2.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]]
Velocity Jacobian (can be multiplied with qdot to obtain end-

In [64]:
import sympy
import numpy as np

def derive_eom(no_of_links,D,V):
  phi = [0]*no_of_links
  c = [[[0]*no_of_links]*no_of_links]*no_of_links
  for i in range(n):
    for j in range(n):
      for k in range(n):
        c[i][j][k] = 0.5*(sp.diff(D[k][j],q[i])+sp.diff(D[k][i],q[j])-sp.diff(D[i][j],q[k]))
  torque = [0]*n
  element_d = 0
  element_christoffel = 0
  for k in range(n):
    phi[k] = sp.diff(V,q[k])
    for j in range(n):
      element_d = element_d + D[k][j] * qddot[j]
      for i in range(n):
        element_christoffel = element_christoffel + c[i][j][k]*qdot[i]*qdot[j]
    torque[k] = element_d + element_christoffel + phi[k]
  print("The equations of motion are:")
  for i in range(n):
    print('torque'+str(i+1)+'='+str(torque[i]))
q = []
qdot = []
qddot = []
no_of_links = 2
for i in range(no_of_links):
  q.append(sp.symbols('q'+str(i+1)))
  qdot.append(sp.symbols('qdot'+str(i+1)))
  qddot.append(sp.symbols('qddot'+str(i+1)))
D_matrix = [[10,2*sp.cos(q[1]-q[0])],[4*sp.cos(q[1]-q[0]),4]]
V = 5*sp.sin(q[0]) + 10*(10*sp.sin(q[0])+3*sp.sin(q[1]))
derive_eom(no_of_links,D_matrix,V)

The equations of motion are:
torque1=10*qddot1 + 2*qddot2*cos(q1 - q2) + 2.0*qdot1**2*sin(q1 - q2) + 4.0*qdot1*qdot2*sin(q1 - q2) + 2.0*qdot2**2*sin(q1 - q2) + 105*cos(q1)
torque2=4*qddot1*cos(q1 - q2) + 10*qddot1 + 2*qddot2*cos(q1 - q2) + 4*qddot2 + 2.0*qdot1**2*sin(q1 - q2) + 4.0*qdot1*qdot2*sin(q1 - q2) + 2.0*qdot2**2*sin(q1 - q2) + 30*cos(q2)
